In [ ]:
include("../Engine/Engine.jl")
using .Engine

using StatsBase, DataFrames
using Graphs, SimpleWeightedGraphs
using DataStructures
import PyPlot as plt
import Seaborn as sns
import PlotlyJS

rcParams = plt.PyDict(plt.matplotlib."rcParams")
rcParams["text.usetex"] = false
rcParams["font.family"] = ["sans-serif"]
rcParams["font.serif"] = ["Computer Modern Roman"];

In [ ]:
graphs_JDD, cascades_JDD, df_JDD = load_data("/home/ubuntu/Thesis/Results/JDD_all_users/Skripal/data.jld2")

centrality_ranks_JDD = get_centrality_ranks(graphs_JDD, df_JDD, 0, "Any Edge")
all_centrality_ranks_JDD = get_centrality_ranks_all_edges(graphs_JDD, df_JDD, 0);

In [ ]:
graphs_TE, cascades_TE, df_TE = load_data("/home/ubuntu/Thesis/Results/TE_all_users/Skripal/data.jld2")

centrality_ranks_TE = get_centrality_ranks(graphs_TE, df_TE, 0, "Any Edge")
all_centrality_ranks_TE = get_centrality_ranks_all_edges(graphs_TE, df_TE, 0);

In [ ]:
# graphs_JDD, cascades_JDD, df_JDD = load_data("/home/ubuntu/Thesis/Results/JDD_all_users/Skripal_no_surro/data.jld2")

# centrality_ranks_JDD = get_centrality_ranks(graphs_JDD, df_JDD, 0, "Any Edge")
# all_centrality_ranks_JDD = get_centrality_ranks_all_edges(graphs_JDD, df_JDD, 0);

In [ ]:
# graphs_TE, cascades_TE, df_TE = load_data("/home/ubuntu/Thesis/Results/TE_all_users/Skripal_no_surro/data.jld2")

# centrality_ranks_TE = get_centrality_ranks(graphs_TE, df_TE, 0.04, "Any Edge")
# all_centrality_ranks_TE = get_centrality_ranks_all_edges(graphs_TE, df_TE, 0.04);

In [ ]:
general_ranks = get_general_ranks(Skripal, skripal_dates);

In [ ]:
corr_JDD_TE = correlation_JDD_TE(all_centrality_ranks_JDD, all_centrality_ranks_TE, 50)
show(corr_JDD_TE, allcols=true)

# Table general

In [ ]:
general_ranking = find_max_ranks(general_ranks, 10)
print(latexify(general_ranking))

In [ ]:
show(general_ranking[3], allcols=true)

# Tables JDD

In [ ]:
centrality_ranking_JDD = find_max_ranks(centrality_ranks_JDD, 10)
print(latexify(centrality_ranking_JDD))

In [ ]:
show(centrality_ranking_JDD[3], allcols=true)

In [ ]:
all_centrality_ranking_JDD = find_max_ranks(all_centrality_ranks_JDD, 10)
print(latexify(all_centrality_ranking_JDD))

In [ ]:
show(all_centrality_ranking_JDD[3], allcols=true)

# Tables TE

In [ ]:
centrality_ranking_TE = find_max_ranks(centrality_ranks_TE, 10)
print(latexify(centrality_ranking_TE))

In [ ]:
show(centrality_ranking_TE[3], allcols=true)

In [ ]:
all_centrality_ranking_TE = find_max_ranks(all_centrality_ranks_TE, 10)
print(latexify(all_centrality_ranking_TE))

In [ ]:
show(all_centrality_ranking_TE[3], allcols=true)

In [ ]:
println(findfirst(ranking1.outdegree_rank .== "Ian56789"))
println(findfirst(ranking1.betweenness_rank .== "Ian56789"))

In [ ]:
general_ranks[1][general_ranks[1].username .== "Ian56789", :]

In [ ]:
rankingJDD = find_max_ranks(all_centrality_ranks_JDD, 15)
rankingTE = find_max_ranks(all_centrality_ranks_TE, 10);

In [ ]:
show(rankingJDD[1], allcols=true)

# Correlation matrices

In [ ]:
plot_correlation_matrices(general_ranks, centrality_ranks_JDD, 50, save=false, foldername="/home/ubuntu/Thesis/Figures/Skripal/JDD_all_users/")

In [ ]:
plot_correlation_matrices(general_ranks, centrality_ranks_TE, 50, save=false, foldername="/home/ubuntu/Thesis/Figures/Skripal/TE_all_users/")

: 

# Edge types

In [ ]:
"""
Plot the different edge types count or proportion, for different partitions and/or datasets.
"""
function plot_edge_types2(graphs, dfs, cuttoffs; y::String = "count", log::Bool = true, save::Bool = false, filename = nothing, loc="best", kwargs...)

    if save && isnothing(filename)
        throw(ArgumentError("You must provide a filename if you want to save the figure."))
    end

    acceptable_y = ["count", "count_normalized", "proportion"]
    if !(y in acceptable_y)
        throw(ArgumentError("y must be one of $acceptable_y."))
    end

    data = edge_types(graphs, dfs, cuttoffs)

    plt.figure()
    sns.barplot(data, x="edge_type", y=y, hue="partition", saturation=1, zorder=2; kwargs...)
    plt.xlabel("Edge type")
    if y == "count_normalized"
        plt.ylabel("Normalized count of edges")
    else
        plt.ylabel(uppercasefirst(y) * " of edges")
    end
    plt.legend(loc=loc)
    plt.grid(true, which="major", axis="y", zorder=0)
    if log
        plt.yscale("log")
        plt.grid(true, which="minor", axis="y", zorder=0, alpha=0.4)
    end
    if save
        mkpath(dirname(filename))
        plt.savefig(filename, bbox_inches="tight", dpi=400)
    end
    return plt.gcf()

end

In [ ]:
plot_edge_types2(graphs_JDD, df_JDD, 0; y="count", hue_order=["Before", "During", "After"],
    save=true, filename="/home/ubuntu/Thesis/Figures/Skripal/JDD_all_users/edge_type_count.pdf", log=true);

In [ ]:
plot_edge_types2(graphs_TE, df_TE, 0; y="count", hue_order=["Before", "During", "After"],
    save=true, filename="/home/ubuntu/Thesis/Figures/Skripal/TE_all_users/edge_type_count.pdf", log=true);

In [ ]:
plot_edge_types2(graphs_JDD, df_JDD, 0; y="count_normalized", hue_order=["Before", "During", "After"],
    save=true, filename="/home/ubuntu/Thesis/Figures/Skripal/JDD_all_users/edge_type.pdf", log=true);

In [ ]:
plot_edge_types2(graphs_TE, df_TE, 0; y="count_normalized", hue_order=["Before", "During", "After"],
    save=true, filename="/home/ubuntu/Thesis/Figures/Skripal/TE_all_users/edge_type.pdf", log=false);

# Cascades reach

In [ ]:
plot_actors_per_level(cascades_JDD, df_JDD, save=true, filename="/home/ubuntu/Thesis/Figures/Skripal/JDD_all_users/actor_level.pdf", inner_spacing=0);

In [ ]:
plot_actors_per_level(cascades_TE, df_TE, save=true, filename="/home/ubuntu/Thesis/Figures/Skripal/TE_all_users/actor_level.pdf", inner_spacing=0);

# Cascades

In [ ]:
cascades[3]
PlotlyJS.plot(plot_cascade_sankey(cascades[3][6], df)...)
# PlotlyJS.savefig(PlotlyJS.plot(plot_cascade_sankey(cascades[3][5], df)...), "/home/ubuntu/Thesis/Figures/Skripal/TE_all_users/cascade.pdf")